## Intro
This notebook will cover the mathematical details of a multilayer neural network. This will include feedforward computation and gradient calculation using the backpropagation algorithm

## Notation
Suppose we have an $L$-layer neural network with a single training example. Let $M_{\ell}$ be the number of nodes in layer $\ell$. Let $n^{\ell}_i$ be neuron number $i$ in layer $\ell$. Let $b^{\ell}_i$ be the bias weight of neuron $n^{\ell+1}_i$. Let $\Theta^{\ell}_{i,j}$ be the weight connecting $n^{\ell}_i$ to $n^{\ell+1}_j$, making $\Theta^{\ell}$ an $M_{\ell} \times M_{\ell+1}$ matrix. Let $a^{\ell}_i$ be the activation of $n^{\ell}_i$. Let $g$ be the nonlinear activation function for all nodes in the network. Let $z^{\ell}_i$ be the sum of all inputs into neuron $n^{\ell}_i$, such that $a^{\ell}_i = g(z^{\ell}_i)$. In this notebook I will treat $z^{\ell}$, $a^{\ell}$, $b^{\ell}$ and other layer-associated vectors ar *row vectors*, which makes more sense to me when we generalize to large numbers of training examples. This may differ from notations used elsewhere.

Generalizing to $N$ training examples, $Z^{\ell}$ and $A^{\ell}$ become $N \times M_{\ell}$ matrices representing the activations of each neuron on layer $\ell$ for each training example.

## Feedforward

Prior to nonlinear activation, the input to a certain node is just a weighted sum of activaitons of the previous layer. It follows that $Z^{\ell+1} = A^{\ell}\Theta^{\ell} + \mathbf{1}_{N}b^{\ell}$ where $\mathbf{1}_N$ is a $N \times 1$ column vector of ones.
Adding our nonlinear activation, we get that $A^{\ell+1} = g(A^{\ell}\Theta^{\ell} + \mathbf{1}_{N}b^{\ell})$ where $g$ is applied element-wise to each entry. If we let $A^{1}$ be the input ot the network, then we can use this recursion to compute $A^{L}$, which is the network output.

In [ ]:
import numpy as np
import activations as act

class WeightLayer:

    def __init__(self, weights, biases, activ=act.relu, activ_prime=act.relu_prime):
        # weights[i,j] is a weight mapping from neuron i in layer l to j in layer l+1

        self.num_inputs = weights.shape[0]
        self.num_outputs = weights.shape[1]
        self.activ = activ
        self.activ_prime = activ_prime
        self.weights = weights
        self.biases = biases

    def compute(self, A):  # matrix with inputs from different examples stacked vertically
        return self.activ(A @ self.weights + self.biases)

    def set_weights(self, new):
        self.weights = new

    def get_weights(self):
        return self.weights


class NeuralNet:

    def __init__(self):
        self.layers = []
        self.num_layers = 0  # input layer by default

    def add_layer(self, layer):
        self.layers.append(layer)
        self.num_layers += 1

    def compute(self, X):
        activations = np.empty(self.num_layers + 1, dtype=object)
        activations[0] = X
        curr_input = X
        for l in range(self.num_layers):
            curr_input = self.layers[l].compute(curr_input)
            activations[l+1] = curr_input
        return activations

## Gradient Computation and Backpropagation

While feedforward computation seems straightforward, computing the gradient of a loss function with respect to the weights of the network is significantly more complex. For clarity, we will start by considering the case of a single training example. Supporse we have some loss $C$ which acts on the network output. We define $\delta^{\ell}_{i} = \frac{\partial C}{\partial z^{\ell}_{i}}$. The importance of this particular derivative will become clear momentarily.

Define the vector-valued function $f(z^{\ell}_i) = [z^{\ell+1}_1, z^{\ell+1}_2,...,z^{\ell+1}_{M_{\ell+1}}]$, assuming $z^{\ell}_{j\neq i}$ are all constants.

Notice $\delta^{\ell}_{i} = \frac{\partial C}{\partial z^{\ell}_{i}} = \frac{\partial C\left(z^{\ell+1}_1,z^{\ell+1}_2,...,z^{\ell+1}_{M_{\ell+1}}\right)}{\partial z^{\ell}_{i}} = \frac{\partial C(f(z^{\ell}_i))}{\partial z^{\ell}_{i}}$, since the network output is fully determined by $f(z^{\ell}_i)$

Applying the chain rule for Jacobian matrices gives 
$$\delta^{\ell}_{i} = [D C\left(z^{\ell+1}_1,z^{\ell+1}_2,...,z^{\ell+1}_{M_{\ell+1}}\right)][D f(z^{\ell}_i)] = [\frac{\partial C}{z^{\ell+1}_1}, \frac{\partial C}{z^{\ell+1}_2}, ..., \frac{\partial C}{z^{\ell+1}_{M_{\ell+1}}}]\begin{bmatrix} \Theta^{\ell}_{i,1}g'(z^{\ell}_i)\\\Theta^{\ell}_{i,2}g'(z^{\ell}_i)\\\vdots\\\Theta^{\ell}_{i,M_{\ell+1}}g'(z^{\ell}_i)\end{bmatrix} 
= g'(z^{\ell}_i) \sum_{j=1}^{M_{\ell+1}} \delta^{\ell+1}_{j}\Theta^{\ell}_{i,j}
= [\delta^{\ell+1} (\Theta^{\ell})^{T}]_i g'(z^{\ell}_i)$$
Therefore we can compute the entire vector using:
$\delta^{\ell} = \delta^{\ell+1} (\Theta^{\ell})^{T} .* g'(z^{\ell})$.

When we have multiple training examples, we can just stack the row vectors from each training example on top of eachother. If $\Delta^{\ell}$ is a $N \times M_{\ell}$ matrix of $\delta$ values for each training example, then $\Delta^{\ell} = \Delta^{\ell+1} (\Theta^{\ell})^{T} .* g'(Z^{\ell})$.

Remember that our end goal is to compute the gradient of the loss with respect to the the individual weights of the network. Notice:
$$\frac{\partial C}{\partial \Theta^{\ell}_{i,j}} = \frac{\partial C}{\partial z^{\ell+1}_{j}}\frac{\partial z^{\ell+1}_j}{\Theta^{\ell}_{i,j}}
= \delta^{\ell+1}_j a^{\ell}_i$$

For N training examples this becomes $\frac{\partial C}{\partial \Theta^{\ell}_{i,j}} = \sum_{k=1}^{N} \Delta^{\ell+1}_{k,j} A^{\ell}_{k,i}$ by the linearity of the derivative, assuming the cost function just adds across training examples.

$\frac{\partial C}{\partial \Theta^{\ell}_{i,j}} = \sum_{k=1}^{N} \Delta^{\ell+1}_{k,j} A^{\ell}_{k,i} = \sum_{k=1}^{N} (A^{\ell})^{T}_{i,k}\Delta^{\ell+1}_{k,j}
= [(A^{\ell})^{T} \Delta^{\ell+1}]_{i,j}$

Therefore if we let $G^{\ell}$ be matrix of derivatives with respect to the weights in $\Theta^{\ell}$, then $G^{\ell} = (A^{\ell})^{T} \Delta^{\ell+1}$ 

Now to derive the gradient with respect to biases:
$$\frac{\partial C}{\partial b^{\ell}_{i}} = \frac{\partial C}{\partial z^{\ell+1}_{i}}\frac{\partial z^{\ell+1}_i}{b^{\ell}_{i}}
= \delta^{\ell+1}_i$$

For multiple training examples, it follows that the bias gradient $G^{\ell}_{b} = \mathbf{1}^{T}_{N}\Delta^{\ell}$ where $\mathbf{1}^{T}_{N}$ is a row vector of $N$ ones. Equivalently, $G^{\ell}_{b}$ can be found by summing the columns of $\Delta^{\ell}$.

### Recap

To recap, we can compute the values of $\Delta^{\ell}$ by using the recurrence:
$$\Delta^{\ell} = \Delta^{\ell+1} (\Theta^{\ell})^{T} .* g'(Z^{\ell})$$
Note that we must still calculate the base case $\Delta^{L}$ using the gradient of our particular loss function.<br>

We then calculate the actual gradients using:
$$G^{\ell} = (A^{\ell})^{T} \Delta^{\ell+1}\\
G^{\ell}_{b} = \mathbf{1}^{T}_{N}\Delta^{\ell}$$

The next cell adds this functionality to our neural network class

In [ ]:
    def backpropagate(self, X, Y, cost_gradient):
        A = self.compute(X)  # activations of all layers
        Delta = np.empty(self.num_layers, dtype=object)
        Grad = np.empty(self.num_layers, dtype=object)

        Delta[self.num_layers-1] = cost_gradient(A[-1], Y)  # N x K matrix Delta_L
        Grad[self.num_layers-1] = A[-2].T @ Delta[-1]

        for l in reversed(range(1, self.num_layers)):  # iterate backwards to second layer
            Z_l = A[l-1] @ self.layers[l-1].weights + self.layers[l-1].biases
            Delta[l-1] = (Delta[l] @ self.layers[l].weights.T) * self.layers[l].activ_prime(Z_l)
            Grad[l-1] = A[l-1].T @ Delta[l-1]  # delta indices are down-shifted by 1

        Grad_bias = np.array([np.sum(delt, axis=0) for delt in Delta])
        return np.concatenate((Grad, Grad_bias))

